In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/Colab Notebooks/Thesis Project/

!pip install transformers
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

Mounted at /content/drive/
[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Thesis Project/'
/content


In [ ]:
print(torch.__version__)

2.5.0+cu121


In [ ]:
# Load the data
df = pd.read_excel('/content/drive/MyDrive/nlp project dataset/BSMDD_v3_textcleaned - 21K.xlsx')

df['label'] = df['label'].map({'Non Depressive': 0, 'Depressive': 1})
# Drop rows with any missing values
df = df.dropna(subset=['text', 'label'])
# drop rows that have identical values in all columns
df.drop_duplicates(inplace=True)
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    0 non-null      object 
 1   label   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 0.0+ bytes
None


,text,label


In [ ]:
# Check the column names in the DataFrame
print("Columns in DataFrame:", df.columns)

# If 'word_length' or 'text_length' columns are missing, verify and correct them accordingly
if 'word_length' not in df.columns or 'text_length' not in df.columns:
    print("Please make sure 'word_length' and 'text_length' columns exist in the DataFrame.")
else:
    # Proceed with calculations and visualization if columns exist
    word_length_mean = df['word_length'].mean()
    word_length_median = df['word_length'].median()
    word_length_mode = df['word_length'].mode()[0]

    text_length_mean = df['text_length'].mean()
    text_length_median = df['text_length'].median()
    text_length_mode = df['text_length'].mode()[0]

    # Calculate skewness
    word_length_skewness = df['word_length'].skew()
    text_length_skewness = df['text_length'].skew()

    # Visualization
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    sns.histplot(data=df, x='word_length', kde=True)
    plt.title('Distribution of Word Length')

    plt.subplot(1, 2, 2)
    sns.histplot(data=df, x='text_length', kde=True)
    plt.title('Distribution of Text Length')

    plt.tight_layout()
    plt.show()

    # Print results
    print('Word Length:')
    print('Mean:', word_length_mean)
    print('Median:', word_length_median)
    print('Mode:', word_length_mode)
    print('Skewness:', word_length_skewness)

    print('\nText Length:')
    print('Mean:', text_length_mean)
    print('Median:', text_length_median)
    print('Mode:', text_length_mode)
    print('Skewness:', text_length_skewness)


Columns in DataFrame: Index(['text', 'label'], dtype='object')
Please make sure 'word_length' and 'text_length' columns exist in the DataFrame.


In [ ]:
# Calculate the length of each text
df['text_length'] = df['text'].str.len()

# Calculate max, min, mean, and mode with a check for mode's availability
max_length = df['text_length'].max()
min_length = df['text_length'].min()
mean_length = df['text_length'].mean()

# Check if mode exists before accessing it
mode_length = df['text_length'].mode()
mode_length = mode_length[0] if not mode_length.empty else None

# Print the results
print("Max Length:", max_length)
print("Min Length:", min_length)
print("Mean Length:", mean_length)
print("Mode Length:", mode_length if mode_length is not None else "No mode available")


Max Length: nan
Min Length: nan
Mean Length: nan
Mode Length: No mode available


### Define Hyperparameter

In [ ]:
# Define batch size
BATCH_SIZE = 16
LEARNING_RATE = 0.1

DROPOUT = 0.1

# Training loop
NUM_EPOCHS = 10

# Define number of folds for cross-validation
NUM_FOLDS = 5

# Momentum -> hyperparameter of Stochastic Gradient Descent (SGD) optimizer
MOMENTUM = 0.9


In [ ]:
# Load the data
df = pd.read_excel('/content/drive/MyDrive/nlp project dataset/BSMDD_v3_textcleaned - 21K.xlsx')
print("Data loaded, shape:", df.shape)  # Check if data loads correctly

# Mapping labels
if 'label' in df.columns:
    df['label'] = df['label'].map({'Non Depressive': 0, 'Depressive': 1})
    print("Label mapping done.")
else:
    print("Column 'label' not found in DataFrame.")

# Drop rows with any missing values in 'text' and 'label'
df = df.dropna(subset=['text', 'label'])
print("After dropping missing values, shape:", df.shape)

# Drop duplicate rows
df.drop_duplicates(inplace=True)
print("After dropping duplicates, shape:", df.shape)

# Keep only 'text' and 'label' columns
columns_to_drop = [col for col in df.columns if col not in ['text', 'label']]
df.drop(columns=columns_to_drop, inplace=True)
print("After dropping unnecessary columns, shape:", df.shape)

# Check DataFrame info and label counts
print(df.info())
label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

# Display a sample if DataFrame is not empty
if not df.empty:
    print(df.sample(n=10))
else:
    print("DataFrame is empty after processing.")


Data loaded, shape: (21910, 2)
Label mapping done.
After dropping missing values, shape: (0, 2)
After dropping duplicates, shape: (0, 2)
After dropping unnecessary columns, shape: (0, 2)
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    0 non-null      object 
 1   label   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 0.0+ bytes
None
Label Details:
Series([], Name: count, dtype: int64)
DataFrame is empty after processing.


In [ ]:
# Check the counts of each label
label_counts = df['label'].value_counts()
print("Label counts before sampling:", label_counts)

# Determine the minimum count available for each label
min_count = min(label_counts.get(0, 0), label_counts.get(1, 0))

# Sample only up to the available number of instances for each label
df_label_0 = df[df['label'] == 0].sample(n=min_count, random_state=42)
df_label_1 = df[df['label'] == 1].sample(n=min_count, random_state=42)

# Concatenate the sampled dataframes
df = pd.concat([df_label_0, df_label_1], ignore_index=True)

# Display info and label distribution
print(df.info())
label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

# Display random samples
if not df.empty:
    print(df.sample(n=10))
else:
    print("DataFrame is empty after sampling.")


Label counts before sampling: Series([], Name: count, dtype: int64)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    0 non-null      object 
 1   label   0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 124.0+ bytes
None
Label Details:
Series([], Name: count, dtype: int64)
DataFrame is empty after sampling.


In [ ]:
X = df['text']
y = df['label']
print(X.info())
print(y.info())

<class 'pandas.core.series.Series'>
RangeIndex: 0 entries
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
0 non-null      object
dtypes: object(1)
memory usage: 124.0+ bytes
None
<class 'pandas.core.series.Series'>
RangeIndex: 0 entries
Series name: label
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 124.0 bytes
None


In [ ]:
torch.cuda.empty_cache()

# Define the BERT model architecture

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask)[1]  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits


# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and verify data for X and y
try:
    # Replace with actual data loading code
    # For example:
    # df = pd.read_csv('path_to_your_dataset.csv')
    # X = df['text_column_name']
    # y = df['label_column_name']

    # Check if X and y are properly loaded and contain data
    if 'X' in globals() and 'y' in globals() and len(X) > 0 and len(y) > 0:
        # Convert X to a list of strings if it's not already
        X = list(X)

        # Load the BERT tokenizer and encode the text data
        tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        X_encodings = tokenizer(X, truncation=True, padding=True, max_length=512, return_tensors="pt")

        # Convert labels to tensor
        y_tensor = torch.tensor(y.values)
    else:
        print("Error: X and/or y are not defined or are empty. Please ensure the data is correctly loaded.")
        raise ValueError("X or y is empty")

except Exception as e:
    print(f"An error occurred during data loading or tokenization: {e}")
    raise

# Initialize the StratifiedKFold
NUM_FOLDS = 5  # Define number of folds
BATCH_SIZE = 16  # Set batch size
LEARNING_RATE = 1e-3  # Set learning rate
MOMENTUM = 0.9  # Set momentum
NUM_EPOCHS = 3  # Set number of epochs

skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_encodings['input_ids'], y_tensor), 1):
    print(f"Processing Fold: {fold}")

    # Split data into train and validation sets
    X_train_fold = {key: X_encodings[key][train_index] for key in X_encodings}
    X_val_fold = {key: X_encodings[key][val_index] for key in X_encodings}
    y_train_fold = y_tensor[train_index]
    y_val_fold = y_tensor[val_index]

    # Create PyTorch datasets
    train_dataset = TensorDataset(X_train_fold['input_ids'], X_train_fold['attention_mask'], y_train_fold)
    val_dataset = TensorDataset(X_val_fold['input_ids'], X_val_fold['attention_mask'], y_val_fold)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # Initialize the BERT model
    bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
    hidden_dim = bert_model.config.hidden_size
    num_classes = 2  # Adjust this as per your actual number of classes
    model = BERTClassifier(bert_model, hidden_dim, num_classes)
    model.to(DEVICE)

    # Define optimizer and loss function
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
    criterion = nn.CrossEntropyLoss()

    best_accuracy = 0.0
    best_model_path = f'best_model_fold{fold}.pth'

    # Define lists to store train and validation metrics per epoch
    train_losses_per_epoch = []
    train_accuracies_per_epoch = []
    val_losses_per_epoch = []
    val_accuracies_per_epoch = []

    for epoch in range(NUM_EPOCHS):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(DEVICE)
            attention_mask = attention_mask.to(DEVICE)
            labels = labels.to(DEVICE)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            _, predicted = torch.max(logits, 1)

            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * input_ids.size(0)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        train_accuracy = 100 * train_correct / train_total

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = batch
                input_ids = input_ids.to(DEVICE)
                attention_mask = attention_mask.to(DEVICE)
                labels = labels.to(DEVICE)

                logits = model(input_ids, attention_mask)
                _, predicted = torch.max(logits, 1)

                loss = criterion(logits, labels)

                val_loss += loss.item() * input_ids.size(0)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total

        train_losses_per_epoch.append(train_loss / train_total)
        train_accuracies_per_epoch.append(train_accuracy)
        val_losses_per_epoch.append(val_loss / val_total)
        val_accuracies_per_epoch.append(val_accuracy)

        print('Epoch: {}/{} | Train Loss: {:.4f} | Train Accuracy: {:.2f}% | Val Loss: {:.4f} | Val Accuracy: {:.2f}%'.format(
            epoch + 1, NUM_EPOCHS, train_loss / train_total, train_accuracy, val_loss / val_total, val_accuracy))

        # Save the best model based on validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model, best_model_path)
            print("Best model saved!")

    print("Training completed for Fold:", fold)

    # Load the best model for evaluation if needed
    best_model = torch.load(best_model_path)
    best_model.to(DEVICE)
    best_model.eval()

    # You can proceed with further evaluation or inference here


Error: X and/or y are not defined or are empty. Please ensure the data is correctly loaded.
An error occurred during data loading or tokenization: X or y is empty


ValueError: X or y is empty

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the test data
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             torch.tensor(y_test.values))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Load the best model for evaluation
best_model = torch.load(best_model_path)
best_model.to(DEVICE)
best_model.eval()

test_loss = 0.0
test_correct = 0
test_total = 0
predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(DEVICE)
        attention_mask = attention_mask.to(DEVICE)
        labels = labels.to(DEVICE)

        logits = best_model(input_ids, attention_mask)
        _, predicted = torch.max(logits, 1)

        loss = criterion(logits, labels)

        test_loss += loss.item() * input_ids.size(0)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        predictions.extend(predicted.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
test_loss = test_loss / test_total

# Calculate precision, recall, and F1 score
precision = precision_score(y_test.values, predictions)
recall = recall_score(y_test.values, predictions)
f1 = f1_score(y_test.values, predictions)

print('Test Loss: {:.4f} | Test Accuracy: {:.2f}%'.format(test_loss, test_accuracy))
print('Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(precision, recall, f1))


In [ ]:
# Plot the validation and test accuracy
plt.plot(range(1, NUM_EPOCHS + 1), val_accuracies_per_epoch, label='Validation Accuracy')
plt.axhline(y=test_accuracy, color='r', linestyle='--', label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation and Test Accuracy')
plt.legend()
plt.show()

# Plot the validation and test loss
plt.plot(range(1, NUM_EPOCHS + 1), val_losses_per_epoch, label='Validation Loss')
plt.axhline(y=test_loss, color='r', linestyle='--', label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Validation and Test Loss')
plt.legend()
plt.show()


In [ ]:
# Define the BERT model architecture
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(DROPOUT)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return torch.sigmoid(logits)


# Set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the BERT tokenizer and encode the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
X_encodings = tokenizer(list(X), truncation=True, padding=True, max_length=512)

# Convert labels to tensor
y_tensor = torch.tensor(y.values)

# Initialize the StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_encodings['input_ids'], y_tensor), 1):
    print(f"Processing Fold: {fold}")

    # Convert X_encodings to a list of tuples
    X_encodings_list = list(X_encodings.items())

    # Split data into train and validation sets
    X_train_fold = {key: [value[i] for i in train_index] for key, value in X_encodings_list}
    X_val_fold = {key: [value[i] for i in val_index] for key, value in X_encodings_list}
    y_train_fold = y_tensor[train_index]
    y_val_fold = y_tensor[val_index]


    # Create PyTorch datasets
    train_dataset = TensorDataset(torch.tensor(X_train_fold['input_ids']),
                                  torch.tensor(X_train_fold['attention_mask']),
                                  torch.tensor(y_train_fold))
    val_dataset = TensorDataset(torch.tensor(X_val_fold['input_ids']),
                                torch.tensor(X_val_fold['attention_mask']),
                                torch.tensor(y_val_fold))

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


Processing Fold: 1


<ipython-input-9-9705a8b77c1d>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_train_fold))
<ipython-input-9-9705a8b77c1d>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y_val_fold))


Processing Fold: 2
Processing Fold: 3
Processing Fold: 4
Processing Fold: 5


In [ ]:
torch.cuda.empty_cache()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask)[1]  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits

# Set the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BERT model
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')

hidden_dim = bert_model.config.hidden_size
num_classes = 2

model = BERTClassifier(bert_model, hidden_dim, num_classes).to(DEVICE)



# Freeze BERT layer
for param in model.bert_model.parameters():
    param.requires_grad = False

# Unfreeze output layer
for param in model.fc.parameters():
    param.requires_grad = True


# Define the optimizer and loss function
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE, momentum=MOMENTUM)
criterion = nn.BCEWithLogitsLoss()

# Define the optimizer and loss function
# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
# criterion = nn.CrossEntropyLoss()

best_accuracy = 0.0

# Define lists to store metrics for each fold
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []
fold_val_predictions = []
fold_val_labels = []

# Define lists to store train and validation metrics per epoch
train_losses_per_epoch = []
train_accuracies_per_epoch = []
val_losses_per_epoch = []
val_accuracies_per_epoch = []

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch in train_loader:
      input_ids, attention_mask, labels = batch
      input_ids = input_ids.to(DEVICE)
      attention_mask = attention_mask.to(DEVICE)
      labels = labels.to(DEVICE)

      optimizer.zero_grad()

      logits = model(input_ids, attention_mask)
      _, predicted = torch.max(logits, 1)

      loss = criterion(logits, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item() * input_ids.size(0)
      train_total += labels.size(0)
      train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch in val_loader:
          input_ids, attention_mask, labels = batch
          input_ids = input_ids.to(DEVICE)
          attention_mask = attention_mask.to(DEVICE)
          labels = labels.to(DEVICE)

          logits = model(input_ids, attention_mask)
          _, predicted = torch.max(logits, 1)

          loss = criterion(logits, labels)

          val_loss += loss.item() * input_ids.size(0)
          val_total += labels.size(0)
          val_correct += (predicted == labels).sum().item()

          fold_val_predictions.extend(predicted.cpu().numpy())
          fold_val_labels.extend(labels.cpu().numpy())

    val_accuracy = 100 * val_correct / val_total

    fold_train_losses.append(train_loss / train_total)
    fold_train_accuracies.append(train_accuracy)
    fold_val_losses.append(val_loss / val_total)
    fold_val_accuracies.append(val_accuracy)

    train_losses_per_epoch.append(train_loss / train_total)
    train_accuracies_per_epoch.append(train_accuracy)
    val_losses_per_epoch.append(val_loss / val_total)
    val_accuracies_per_epoch.append(val_accuracy)

    print('Epoch: {}/{} | Train Loss: {:.4f} | Train Accuracy: {:.2f}% | Val Loss: {:.4f} | Val Accuracy: {:.2f}%'.format(
        epoch + 1, NUM_EPOCHS, train_loss / train_total, train_accuracy, val_loss / val_total, val_accuracy))

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')


# Calculate performance metrics
threshold = 0.5  # Adjust the threshold based on your task and preferences
fold_val_predictions_binary = [1 if pred >= threshold else 0 for pred in fold_val_predictions]
fold_val_labels_binary = [1 if label >= threshold else 0 for label in fold_val_labels]

average_accuracy = accuracy_score(fold_val_labels_binary, fold_val_predictions_binary)
average_precision = precision_score(fold_val_labels_binary, fold_val_predictions_binary)
average_recall = recall_score(fold_val_labels_binary, fold_val_predictions_binary)
average_f1_score = f1_score(fold_val_labels_binary, fold_val_predictions_binary)

# Print average metrics across folds
print('Average Metrics Across Folds:')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(
    average_accuracy, average_precision, average_recall, average_f1_score))

# Plotting the train and validation accuracy and loss graphs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses_per_epoch, label='Train Loss')
plt.plot(val_losses_per_epoch, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies_per_epoch, label='Train Accuracy')
plt.plot(val_accuracies_per_epoch, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the BERT classifier model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask)[1]  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits

# Set the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the BERT model
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')


hidden_dim = bert_model.config.hidden_size
num_classes = 2
#bert_model = torch.compile(bert_model)
model = BERTClassifier(bert_model, hidden_dim, num_classes).to(DEVICE)
model = torch.compile(model)

# Define the optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
criterion = nn.CrossEntropyLoss()

best_accuracy = 0.0

# Freeze BERT layer
for param in model.bert_model.parameters():
    param.requires_grad = False

# Unfreeze output layer
for param in model.fc.parameters():
    param.requires_grad = True

# Define lists to store metrics for each fold
fold_train_losses = []
fold_train_accuracies = []
fold_val_losses = []
fold_val_accuracies = []
fold_val_predictions = []
fold_val_labels = []

# Define lists to store train and validation metrics per epoch
train_losses_per_epoch = []
train_accuracies_per_epoch = []
val_losses_per_epoch = []
val_accuracies_per_epoch = []

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch in train_loader:
      input_ids, attention_mask, labels = batch
      input_ids = input_ids.to(DEVICE)
      attention_mask = attention_mask.to(DEVICE)
      labels = labels.to(DEVICE)

      optimizer.zero_grad()

      logits = model(input_ids, attention_mask)
      _, predicted = torch.max(logits, 1)

      loss = criterion(logits, labels)
      loss.backward()
      optimizer.step()

      train_loss += loss.item() * input_ids.size(0)
      train_total += labels.size(0)
      train_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * train_correct / train_total

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for batch in val_loader:
          input_ids, attention_mask, labels = batch
          input_ids = input_ids.to(DEVICE)
          attention_mask = attention_mask.to(DEVICE)
          labels = labels.to(DEVICE)

          logits = model(input_ids, attention_mask)
          _, predicted = torch.max(logits, 1)

          loss = criterion(logits, labels)

          val_loss += loss.item() * input_ids.size(0)
          val_total += labels.size(0)
          val_correct += (predicted == labels).sum().item()

          fold_val_predictions.extend(predicted.cpu().numpy())
          fold_val_labels.extend(labels.cpu().numpy())

    val_accuracy = 100 * val_correct / val_total

    fold_train_losses.append(train_loss / train_total)
    fold_train_accuracies.append(train_accuracy)
    fold_val_losses.append(val_loss / val_total)
    fold_val_accuracies.append(val_accuracy)

    train_losses_per_epoch.append(train_loss / train_total)
    train_accuracies_per_epoch.append(train_accuracy)
    val_losses_per_epoch.append(val_loss / val_total)
    val_accuracies_per_epoch.append(val_accuracy)

    print('Epoch: {}/{} | Train Loss: {:.4f} | Train Accuracy: {:.2f}% | Val Loss: {:.4f} | Val Accuracy: {:.2f}%'.format(
        epoch + 1, NUM_EPOCHS, train_loss / train_total, train_accuracy, val_loss / val_total, val_accuracy))

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')

# Calculate performance metrics
average_accuracy = accuracy_score(fold_val_labels, fold_val_predictions)
average_precision = precision_score(fold_val_labels, fold_val_predictions)
average_recall = recall_score(fold_val_labels, fold_val_predictions)
average_f1_score = f1_score(fold_val_labels, fold_val_predictions)

# Print average metrics across folds
print('Average Metrics Across Folds:')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1 Score: {:.4f}'.format(
    average_accuracy, average_precision, average_recall, average_f1_score))

# Plotting the train and validation accuracy and loss graphs
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses_per_epoch, label='Train Loss')
plt.plot(val_losses_per_epoch, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies_per_epoch, label='Train Accuracy')
plt.plot(val_accuracies_per_epoch, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[2023-06-17 16:56:13,583] torch._inductor.utils: [WARNING] using triton random, expect difference from eager


Epoch: 1/40 | Train Loss: 0.5583 | Train Accuracy: 71.62% | Val Loss: 0.4268 | Val Accuracy: 81.09%
Epoch: 2/40 | Train Loss: 0.4970 | Train Accuracy: 76.56% | Val Loss: 0.4249 | Val Accuracy: 80.73%
Epoch: 3/40 | Train Loss: 0.4867 | Train Accuracy: 77.50% | Val Loss: 0.4611 | Val Accuracy: 80.86%
Epoch: 4/40 | Train Loss: 0.4704 | Train Accuracy: 78.16% | Val Loss: 0.3855 | Val Accuracy: 83.00%
Epoch: 5/40 | Train Loss: 0.4749 | Train Accuracy: 78.38% | Val Loss: 0.4354 | Val Accuracy: 82.12%
Epoch: 6/40 | Train Loss: 0.4620 | Train Accuracy: 78.83% | Val Loss: 0.3740 | Val Accuracy: 83.48%
Epoch: 7/40 | Train Loss: 0.4610 | Train Accuracy: 78.91% | Val Loss: 0.3909 | Val Accuracy: 82.77%
Epoch: 8/40 | Train Loss: 0.4582 | Train Accuracy: 79.43% | Val Loss: 0.3708 | Val Accuracy: 83.79%
Epoch: 9/40 | Train Loss: 0.4542 | Train Accuracy: 79.27% | Val Loss: 0.3892 | Val Accuracy: 82.61%


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data
df = pd.read_excel('0. All Datasets.xlsx')
df['label'] = df['label'].map({'Non Depressive': 0, 'Depressive': 1})
df = df.dropna(subset=['text', 'label'])
df.drop_duplicates(inplace=True)
columns_to_drop = [col for col in df.columns if col not in ['text', 'label']]
df.drop(columns=columns_to_drop, inplace=True)

label_counts = df['label'].value_counts()
print("Label Details:")
print(label_counts)

# Sample 14,000 instances for each label
df_label_0 = df[df['label'] == 0].sample(n=14000, random_state=42)
df_label_1 = df[df['label'] == 1].sample(n=14000, random_state=42)

# Concatenate the sampled dataframes
df = pd.concat([df_label_0, df_label_1], ignore_index=True)

X = df['text']
y = df['label']

Label Details:
0    16719
1    14976
Name: label, dtype: int64


In [ ]:
# Define the BERT classifier model
class BERTClassifierWrapper(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_classes):
        super(BERTClassifierWrapper, self).__init__()
        self.bert_model = bert_model
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, input_ids, attention_mask):
        output = self.bert_model(input_ids, attention_mask)[1]  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits

    def fit(self, input_ids, attention_mask, labels, criterion, optimizer, num_epochs, batch_size):
        dataset = TensorDataset(input_ids, attention_mask, labels)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        self.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            for i, batch in enumerate(data_loader):
                input_ids_batch, attention_mask_batch, labels_batch = batch
                input_ids_batch = input_ids_batch.to(DEVICE)
                attention_mask_batch = attention_mask_batch.to(DEVICE)
                labels_batch = labels_batch.to(DEVICE)

                optimizer.zero_grad()

                outputs = self(input_ids_batch, attention_mask_batch)
                loss = criterion(outputs, labels_batch)

                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(f"Epoch {epoch+1}/{num_epochs} Loss: {running_loss / len(data_loader)}")

    def predict(self, input_ids, attention_mask):
        dataset = TensorDataset(input_ids, attention_mask)
        data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

        self.eval()
        predictions = []
        with torch.no_grad():
            for batch in data_loader:
                input_ids_batch, attention_mask_batch = batch
                input_ids_batch = input_ids_batch.to(DEVICE)
                attention_mask_batch = attention_mask_batch.to(DEVICE)

                outputs = self(input_ids_batch, attention_mask_batch)
                _, predicted_labels = torch.max(outputs, dim=1)

                predictions.extend(predicted_labels.cpu().numpy())

        return predictions

# Load the BERT tokenizer and encode the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
X_encodings = tokenizer(list(X), truncation=True, padding=True, max_length=512)

# Convert labels to tensor
y_tensor = torch.tensor(y.values)

# Set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define hyperparameters and their possible values for grid search
parameters = {
    'num_epochs': [5, 10],
    'batch_size': [32, 64],
    'learning_rate': [0.001, 0.01],
    'momentum': [0.9, 0.95],
    'dropout': [0.1, 0.2],
}

# Initialize the BERT model
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
hidden_dim = bert_model.config.hidden_size
num_classes = 2

# Create the BERT classifier model
model = BERTClassifierWrapper(bert_model, hidden_dim, num_classes).to(DEVICE)

# Define the optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Define the grid search
grid_search = GridSearchCV(model, parameters, cv=5, scoring='accuracy')

# Run grid search
grid_search.fit(X_encodings['input_ids'], X_encodings['attention_mask'], y_tensor, criterion, optimizer)

# Print the best parameters and score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: ignored